In [1]:
import numpy as np
import csv    #加载csv包便于读取csv文件
import zipfile
import os
import json
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import datetime
import time
from datetime import timedelta

def date_2_timestamp(year,mon,day):
    tempString=str(year)+"-"+str(mon)+"-"+str(day)
    tempTime=time.strptime(tempString, "%Y-%m-%d")
    return time.mktime(tempTime)

def timestamp_2_date(un_time):
    return datetime.datetime.fromtimestamp(un_time)


def timestamp_removeDay_reduce(temp_timestamp):
    temp_date=timestamp_2_date(temp_timestamp)
    temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,1)
    return temp_timestamp

def timestamp_removeDay_add(temp_timestamp):
    temp_date=timestamp_2_date(temp_timestamp)
    if temp_date.month==12:
        return date_2_timestamp(temp_date.year+1,1,1)
    else:
        return date_2_timestamp(temp_date.year,temp_date.month+1,1)
    
def date_removeDay_reduce(temp_date):
    temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,1)
    return timestamp_2_date(temp_timestamp)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# 返回从最小到最大之间经过的时间戳，间隔为一个月
def max_min_2_timestampList(minDate,maxDate):    
    resList=[]
    resMap={}
    for single_date in daterange(minDate, maxDate):
        temp_date=date_removeDay_reduce(single_date)
        temp_timestamp=date_2_timestamp(temp_date.year,temp_date.month,temp_date.day)
        resMap[temp_timestamp]=1 

    for key in resMap.keys():
        resList.append(key)
    
    # print(resList)
    return sorted(resList)

def timestamp_2_string(temp_timestamp):
    temp_date=datetime.datetime.fromtimestamp(temp_timestamp)
    if temp_date.month<10:
        tempMonth="0"+str(temp_date.month)
    else:
        tempMonth=str(temp_date.month)
    return str(temp_date.year)+"-"+tempMonth

In [3]:
minStamp=1541142268
maxStamp=1674844823

minStamp_removeDay=timestamp_removeDay_reduce(minStamp)
maxStamp_removeDay=timestamp_removeDay_reduce(maxStamp)

minDate=datetime.datetime.fromtimestamp( minStamp_removeDay )
maxDate=datetime.datetime.fromtimestamp( maxStamp_removeDay )


# Exchange_defi

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


exchangeList=["UniswapV1","UniswapV2","UniswapV3", "BalancerV1","BalancerV2", "Curve", "SushiSwap", "ShibaSwap"]
# exchangeList=["Curve"]

for item in exchangeList:
    mapPathDir="/mnt/sde1/peilin_defi/data/volume_line/dict/timeWithVolume/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly Volume (USD)', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/volume_line/photo/defi_exchange.pdf",bbox_inches='tight')

# Lending_defi

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


lendingList=["AaveV1","AaveV2", "MakerDAO", 'Compound', "Cream"]

for item in lendingList:
    mapPathDir="/mnt/sde1/peilin_defi/data/volume_line/dict/timeWithVolume/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16,loc=2)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly Volume (USD)', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/volume_line/photo/defi_lending.pdf",bbox_inches='tight')

# Exchange_nft

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
    
    # 消除opensea的下降线
    if labelName=="OpenSea":
        tempMaxList=[]
        for i in range(1,len(list_value)-1):
            front=list_value[i-1]
            cur=list_value[i]
            next=list_value[i+1]
            if cur>front and cur>next:
                tempMaxList.append(i)
                
        print(tempMaxList)
                
        
        temp_list_key=[]
        temp_list_value=[]
        
        for i in range(len(list_key)):
            if i<tempMaxList[-1]:
                temp_list_key.append(list_key[i])
                temp_list_value.append(list_value[i])
                
        list_key=temp_list_key
        list_value=temp_list_value
        
            
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


exchangeList=["OpenSea", "LooksRare", "X2Y2", "Blur"]
for item in exchangeList:
    mapPathDir="/mnt/sde1/peilin_defi/data/volume_line/dict/timeWithVolume/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16,loc=2)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly Volume (USD)', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/volume_line/photo/nft_exchange.pdf",bbox_inches='tight')

# Lending_nft

In [ ]:
def draw(mapPath,labelName):
    with open(mapPath, "rb") as tf:
        exchange_timeMap=pickle.load(tf)

    list_key=[]
    list_value=[]


    for tempTimestamp in max_min_2_timestampList(minDate,maxDate):
        if tempTimestamp not in exchange_timeMap:
            exchange_timeMap[tempTimestamp]=0
        
        list_key.append(timestamp_2_string(tempTimestamp))
        list_value.append( exchange_timeMap[tempTimestamp] )
        
    plt.plot(list_key, list_value,  markersize=20, label=labelName)
    


lendingList=["NFTfiV1","NFTfiV2", "Arcade", "BendDAO"]

for item in lendingList:
    mapPathDir="/mnt/sde1/peilin_defi/data/volume_line/dict/timeWithVolume/"
    mapPath=mapPathDir+item+".map"
    draw(mapPath,item)

list_key_length=len(max_min_2_timestampList(minDate,maxDate))

plt.legend(fontsize=16,loc=2)
plt.xticks(range(1,list_key_length,8),rotation=45,fontsize=20)
plt.yticks(fontsize=20)
plt.yscale('log')
plt.ylabel('Monthly Volume (USD)', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.tight_layout()
plt.grid(True, color='#666666', linestyle = ":", linewidth = "1")

plt.savefig("/mnt/sde1/peilin_defi/data/volume_line/photo/nft_lending.pdf",bbox_inches='tight')